In [ ]:
import os
import sys
import time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as Function
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader
from model.dataset import SMPLTestPairDataset

from model.SkinningNet import SkinningNet
from model.NeuralSkinningPoseTransfer import poseTransfer
from model.utils import PoseTransferLoss, getFacesOneRingIdx, getLaplacianMatrix

from meshplot import plot
import igl
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device:%s"%device)

In [ ]:
batchSize = 16
dataset = SMPLTestPairDataset(complexity = "simple", gender = "male", vertexOrderRandom = False)
dataIter = DataLoader(dataset, num_workers=0, batch_size=batchSize, shuffle=False, drop_last=False)
allDataNUM = len(dataset)

In [ ]:
net = SkinningNet()
net = net.to(device)
net.load_state_dict(torch.load(".\stateDict\skinningNet_finetune_noise.pkl"))
net = net.eval()

In [ ]:
loss = PoseTransferLoss()

F = torch.tensor(dataset.faces).unsqueeze(0).to(device).long().repeat(batchSize, 1, 1)
facesOneRingIdx = torch.tensor(getFacesOneRingIdx(dataset.faces)).to(device).long().unsqueeze(0).repeat(batchSize, 1, 1)

In [ ]:
beginTime = time.time()

PMDLoss = 0.0
ELRLoss = 0.0
completedNum = 0
batchNum = 0
for sV, rV, tV, _ in dataIter:
    sV = sV.to(device).float()
    rV = rV.to(device).float()
    tV = tV.to(device).float()
    
    laplacian = getLaplacianMatrix(sV, F, weight = "cotangent")
    preV, _, _, _, _, _ = poseTransfer(net, sV, facesOneRingIdx, rV, facesOneRingIdx, laplacian, blendShape = "lbs", dLambda = 50)
    PMD, ELR = loss(preV, tV, F)
    PMD = float(PMD)
    ELR = float(ELR)
    
    completedNum += batchSize
    if (np.isnan(PMD) == True) or (np.isnan(ELR) == True):
        continue
    PMDLoss += PMD
    ELRLoss += ELR
    batchNum += 1
    compltePer = completedNum/allDataNUM
    leftTime = ((time.time() - beginTime)/compltePer - (time.time() - beginTime))/60
    sys.stdout.write('\r 训练%i / %i, EMD损失: %f, ELR损失: %f, 当前进度: %0.2f%%, 预计剩余%d分钟'%(completedNum, allDataNUM, PMDLoss/batchNum, ELRLoss/batchNum, compltePer * 100, leftTime))
    sys.stdout.flush()
print("\n")
print("EMD损失: %f, ELR损失: %f, 用时：%0.2f minuetes"%(PMDLoss/batchNum, ELRLoss/batchNum, (time.time() - beginTime) / 60 ))
print("----------------------------------")


In [ ]:
"dqs lambda-20: EMD0.000361, ELR损失: 0.136000"

"lbs, lambda-5: EMD损失: 0.000369, ELR损失: 0.166071"
"lbs, lambda-10:EMD损失: 0.000364, ELR损失: 0.149422"
"lbs, lambda-15:EMD损失: 0.000362, ELR损失: 0.141333"
"lbs, lambda-20:EMD损失: 0.000360, ELR损失: 0.136911"
"lbs, lambda-25:EMD损失: 0.000360, ELR损失: 0.134205"
"lbs, lambda-30:EMD损失: 0.000359, ELR损失: 0.132456"
"lbs, lambda-35:EMD损失: 0.000360, ELR损失: 0.131339"
"lbs, lambda-40:EMD损失: 0.000360, ELR损失: 0.130668"
"lbs, lambda-45:EMD损失: 0.000361, ELR损失: 0.130300"
#plot(preV[0].cpu().numpy(), dataset.faces)
#plot(tV[0].cpu().numpy(), dataset.faces)

In [ ]:
A[np.arange(A.shape[0]), np.arange(A.shape[0])] = -A.sum(axis = 1)

In [ ]:
PMDLoss = 0.0
ELRLoss = 0.0

In [ ]:
(0 if np.isnan(float(ELR)) == True else ELR)

In [ ]:
ELR = np.nan

In [ ]:
ELR